# Preprocessing

## Imports libraries

In [ ]:
import pandas as pd
import os
import pathlib
import ants
from typing import Union, List, Tuple
import multiprocessing
import SimpleITK as sitk
from nipype.interfaces.dcm2nii import Dcm2niix
import numpy as np
from HD_BET.run import run_hd_bet
from nipype.interfaces import fsl
from intensity_normalization.normalize.zscore import ZScoreNormalize

## Step 0: Convert Dicom to Nifti Files
using Dcm2niix, for more information: https://github.com/rordenlab/dcm2niix

Li X, Morgan PS, Ashburner J, Smith J, Rorden C (2016) The first step for neuroimaging data analysis: DICOM to NIfTI conversion. J Neurosci Methods. 264:47-56. doi: 10.1016/j.jneumeth.2016.03.001. PMID: 26945974

In [ ]:
path_to_folder = ""

root = path_to_folder

# goes through the list of files/folders at path_to_folder and only adds directories to dirlist
dirlist = [
    item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item))
]

print(len(dirlist), "subfolders found")

for dir in dirlist:
    converter = Dcm2niix()
    converter.inputs.source_dir = os.path.join(root, dir)
    converter.inputs.compress = "y"
    converter.inputs.merge_imgs = True
    # converter.inputs.compression = 5
    converter.inputs.out_filename = "%i_%t_%n_%d_%f_%p_%q_%s_%z_%m"
    converter.inputs.output_dir = os.path.join(root, dir)
    converter.run()

## Step 1: Get Caseliste
Reads the excel file with all the cases and returns the patient IDs as a list

In [ ]:
path_to_file = ""

cases = pd.read_excel(
        os.path.join(path_to_file, "Cases.xlsx"),
        header=0,
        index_col=None,
        dtype={"ID": str},
    )

caseList = cases.ID.to_list()

## Step 2: Extract Brain